## Introduction
Where did data come from?

The material id needs to be prepended by "mp-". The job is performed below

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import os
# import sys
# module_path = os.path.abspath(os.path.join(''))
# if module_path not in sys.path:
#     sys.path.append(module_path)
    
# print (module_path)

# from utils.editdf import EditFile 

# ed = EditFile()
# ed.generateFile('assets/HalfHeusler.csv', 'assets/Heusler compound.csv')

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# %pip install pymatgen
# %pip install sklearn
# %pip install --upgrade tensorflow
# %pip install keras

import os.path
import pandas as pd

# import pymatgen.core as pg
# from pymatgen.ext.matproj import MPRester
from math import sqrt
import sys

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.ensemble import GradientBoostingRegressor

import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

# dirPath= 'drive/Mydrive/'
dirPath= 'assets/'
# dirPath= '../input/assets/'

In [ ]:
path_to_file = dirPath + 'HalfHeusler.csv'
file_name = dirPath + 'Heusler compound.csv'

if (not os.path.exists(file_name)):
    print ("A formatted csv file is produced\n")
    df = pd.read_csv(path_to_file)
    print (df.head())

    id_list = df['Materials-ID'].to_list()

    n = df.columns[0]
    df.drop(n, axis=1, inplace=True)

    df[n] = ["mp-" + str(x) for x in id_list]
    print (df.head())

    df.to_csv(file_name, index = False)

- Lattice parameters, atomic radii and atomic masses. The atomic radius is calculated value and not the empirical values
- Python library 'pymatgen'

In [ ]:
'''
The Element class is located in the core subpakage inside the periodic_table module. 
The link to the API documentation is below.

    https://pymatgen.org/pymatgen.core.periodic_table.html#pymatgen.core.periodic_table.Element

Similarly the material project APIs are hosted in the following module.

    https://pymatgen.org/pymatgen.ext.matproj.html?highlight=mprester#module-pymatgen.ext.matproj
'''

file_name_train = dirPath + 'Training data.csv'

if (not os.path.exists(file_name_train)):
    print ("Training data will be generated\n")

    heusler_df = pd.read_csv(file_name, header=0, usecols= ['Materials-ID', '4a-site', '4b-site', '4c-site'])
    data = []

    m = MPRester('fmdc9tZK1xE74JOq')
    for idx in heusler_df.index:
        mat_data = m.get_data(heusler_df['Materials-ID'][idx])
        lat = m.get_structure_by_material_id(heusler_df['Materials-ID'][idx])
        
        lat_const = lat.lattice.abc
        mag_moment = sum(lat.site_properties['magmom'])
        
        e1 = pg.Element(heusler_df['4a-site'][idx])
        e2 = pg.Element(heusler_df['4b-site'][idx])
        e3 = pg.Element(heusler_df['4c-site'][idx])

        x1 = e1.atomic_radius
        x2 = e2.atomic_radius
        x3 = e3.atomic_radius
        m1 = e1.atomic_mass
        m2 = e2.atomic_mass
        m3 = e3.atomic_mass
        
        
        x29 = m1+m2+m3
        x30 = x1+x2+x3
        x33 = (x29/3 -m1)
        x34 = (x29/3 -m2)
        x35 = (x29/3 -m3)
        x42 = (x30/3 -x1)
        x43 = (x30/3 -x2)
        x44 = (x30/3 -x3)
        x51 = (x1**2 + x2**2)
        x52 = (x1**2 + x3**2)
            
        data.append((x1,x2,x3,m1,m2,m3,m1**2,m2**2,m3**2,x1**2,x2**2,x3**2,
                     m1**3,m2**3,m3**3,x1**3,x2**3,x3**3,
                     sqrt(m1),sqrt(m2),sqrt(m3),sqrt(x1),sqrt(x2),sqrt(x3),
                     m2/m1, x3/m1, x2/x1,x3/x1, x29,x30,
                     ((m1**2+m2**2+m3**2)/3.)**2, ((x1**2+x2**2+x3**2)/3.)**2,
                     x33, x34, x35, abs(x33), abs(x34), abs(x35), x33**2, x34**2, x35**2,
                     x42, x43, x44, abs(x42), abs(x43), abs(x44), x42**2, x43**2, x44**2,
                     x51, x52,sqrt(x51), sqrt(x52), 
                     mat_data[0]["formation_energy_per_atom"], # mag_moment,
                     lat_const[0], lat_const[1], lat_const[2]
                    ))

    idx = []
    for i in range(1, 55):
        idx.append('x'+str(i))

#     idx.extend(['form_energy_per_atom', 'total_magnetic_moment','a', 'b', 'c'])
    idx.extend(['form_energy_per_atom','a', 'b', 'c'])
    df_train = pd.DataFrame(data, columns = idx)

    df_train.to_csv(file_name_train, index = False)

## Machine Learning
The gradient boosted trees are selected for the regression task. The cross-validation is the first step.



### Cross-Validation
A five fold cross validation will be performed for the better performance of the model.

In [ ]:
df = pd.read_csv(dirPath + 'Training data.csv')
X = df.iloc[:, :-3]
y = df.iloc[:, -1]

k = 5
kf = KFold(n_splits=k)
scoreR2 = []
y_pred = []

gbReg = GradientBoostingRegressor(loss = 'lad', n_estimators=700, max_depth= 18, random_state= 44)

for train_idx, test_idx in kf.split(X):
#     X_train, X_test = X[train_idx, :], X[test_idx, :]
    X_train, X_test = X.iloc[train_idx, :], X.iloc[test_idx, :]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    
    gbReg.fit(X_train, y_train)
    y_pred.extend(gbReg.predict(X_train))
    
    scoreR2.append(gbReg.score(X_test, y_test))
    
print (sum(scoreR2)/k)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

gbReg.fit(X_train, y_train)
print (gbReg.score(X_test, y_test))

0.8575876900175403
0.9153343661809007


## Grid Search
Hyperparameter tuning

In [ ]:
# %pip install numpy
# %pip install matplotlib

max_depth = [14, 16,18,20,22,24]
n_estimators = [100, 400, 700, 1000]
learning_rate = [0.09, 0.1, 0.11, 0.12]
param_grid = dict(max_depth=max_depth, n_estimators= n_estimators)

grid_search = GridSearchCV(gbReg, param_grid, scoring="r2", n_jobs=-1, cv=kf)
grid_result = grid_search.fit(X, y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
print ("\n")

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']

best_model = grid_result.best_estimator_
best_model.fit(X_train, y_train)
print ("The score for best estimator: \n")
print (best_model.score(X_test, y_test))

print ("\n")

for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))
    
# plot results
scores = np.array(means).reshape(len(max_depth), len(n_estimators))
for i, value in enumerate(max_depth):
    plt.plot(n_estimators, scores[i], label='max_depth: ' + str(value))
plt.legend()
plt.xlabel('n_estimators')
plt.ylabel('R2 score')

In [ ]:
scores = np.array(means).reshape(len(n_estimators),len(max_depth))
for i, value in enumerate(n_estimators):
    plt.plot(max_depth, scores[i], label='n_estimators: ' + str(value))
plt.legend()
plt.xlabel('max_depth')
plt.ylabel('R2 score')

'''
    lr =0.1
    max_depth/n_estimators= 16/1000 or 20/700 or 18/700
'''

## Neural Network 

The Boosted Trees performed very well. It was on par with the published paper. A juxtaposition with an artificial neural network would be interesting. The latter must me a better choice. 

ANN is easy to build especially after the introduction of Keras module in Tensorflow. 

### Scaling
Use MinMaxScaler

In [ ]:
# The seed is required for replication of results. Parameters defined.
seed = 1111
n_cols = 54

# df = pd.read_csv('drive/MyDrive/Training data.csv', usecols= ['x1', 'x2', 'x3','x4', 'x5', 'x6','form_energy_per_atom', 'a', 'b', 'c'])
df = pd.read_csv(dirPath + 'Training data.csv')

X = df.iloc[:, :n_cols]
y = df.iloc[:, -3:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

trans = MinMaxScaler()
# trans = StandardScaler()
X_train_scaled = trans.fit_transform(X_train)
X_test_scaled = trans.fit_transform(X_test)
X_scaled = trans.fit_transform(X)
print (X_scaled[0,:])
print (X_train_scaled[0,:])
print (X_test_scaled[0,:])


[4.78260870e-01 3.91304348e-01 5.60000000e-01 1.64504114e-01
 5.14650821e-01 9.39921298e-01 3.48502212e-02 3.14376210e-01
 8.88121115e-01 3.89966555e-01 3.04347826e-01 4.39215686e-01
 6.71244464e-03 1.82203973e-01 8.37248004e-01 3.09616322e-01
 2.29194345e-01 3.31960133e-01 3.18154136e-01 6.31446131e-01
 9.63185617e-01 5.23866112e-01 4.37839988e-01 6.21604719e-01
 1.42343383e-01 1.08332781e-01 5.11347765e-01 4.79681559e-01
 5.90394941e-01 4.71698113e-01 2.38831667e-01 2.50094473e-01
 9.97778804e-01 5.23512429e-01 2.39481808e-01 7.55739640e-01
 1.35823689e-02 5.93217710e-01 5.71444605e-01 2.04636267e-04
 3.54534887e-01 4.72222222e-01 4.88372093e-01 3.78378378e-01
 3.20000000e-01 4.34782609e-02 2.33333333e-01 1.02400000e-01
 1.89035917e-03 5.44444444e-02 2.93858036e-01 4.57718121e-01
 3.64911871e-01 5.41324956e-01]
[0.7826087  0.43478261 1.         0.57105744 0.         0.97680217
 0.33998752 0.         0.95637163 0.72240803 0.34506556 1.
 0.19871264 0.         0.93543885 0.6593259  0.26

## The parameters

#### 6,4500-5500 best so far for epoch 1800-2000

(2, 600)/({nl-3, nn-800/nn-600, ep-2000}: 0.89)

({nl-5, nn-3200, ep-1700}: 0.87)

({nl-5, nn-3500, ep-1800}: 0.906)

5, 2500-3500 epochs 500

7, 2500 by GridSearchCV in kaggle
3, 1200 for one column output for epoch 850

In [20]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

# tolerance_value = 90
n_epochs = 1600
layers = 6
neural_nodes = 4500
lr = 1e-4
out_nodes = 3

# checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_r_square', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# coefficient of determination (R^2) for regression  (only for Keras tensors)
def r_square(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/SS_tot + K.epsilon())

def build_model(n_hidden=layers, n_neurons=neural_nodes, learning_rate=lr, input_shape=[n_cols,]):
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="sigmoid"))
    model.add(keras.layers.Dense(out_nodes))

    model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=learning_rate), metrics=[r_square])

    return model

In [ ]:
# The wrapper is neccessary for GridSearch later in this notebook
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

# The history dictionary is useful for 'loss' plot of the model. In this case, the 'loss' is mean absolute error 
history = keras_reg.fit(X_train_scaled, y_train, epochs=n_epochs,validation_data=(X_test_scaled, y_test),
                        callbacks=callbacks_list)
#               callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=tolerance_value)])

# Test on the holdout data
mse_test = keras_reg.score(X_test_scaled, y_test)
y_pred = keras_reg.predict(X_test_scaled)

plt.figure()
plt.plot(history.history['r_square'])
plt.plot(history.history['val_r_square'])
plt.ylim(0.85,1)
plt.xlim(250)
plt.title("Model co-efficient of determination")
plt.ylabel("R-squared score")
plt.xlabel("Epoch")
plt.legend(["Train", "Test"])
plt.show()

# print ("\n The predicted values (Lattice constants)")
print (y_pred)
print ("\n The mean square error (MSE)")
print (mse_test)
print ("\n The R-square metric is")
print (r_square(tf.convert_to_tensor(value=y_test.values, dtype='float32'), y_pred).numpy())

In [24]:
model = keras.Sequential()
model.add(keras.layers.InputLayer(input_shape=[n_cols,]))
for layer in range(layers):
    model.add(keras.layers.Dense(neural_nodes, activation="sigmoid"))
model.add(keras.layers.Dense(out_nodes))
model.load_weights(filepath)
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=lr), metrics=[r_square])

# Test on the holdout data
scores = model.evaluate(X_scaled, y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
y_pred = model.predict(X_scaled)

print ("\n The predicted values (Lattice constants)")
print (y_pred)
print ("\n The mean square error (MSE)")
print (mse_test)
print ("\n The R-square metric is")
print (r_square(tf.convert_to_tensor(value=y.values, dtype='float32'), y_pred).numpy())

r_square: 91.17%

 The predicted values (Lattice constants)
[[4.554674  4.5615664 4.5623617]
 [4.097783  4.103268  4.103541 ]
 [4.424891  4.430809  4.431807 ]
 [4.674197  4.681325  4.682094 ]
 [4.354044  4.360436  4.3611126]
 [4.5221367 4.528181  4.5292697]
 [4.5182247 4.5243864 4.5254254]
 [4.5197983 4.5259132 4.5269723]
 [3.462201  3.4705153 3.4713852]
 [4.5207486 4.5268345 4.527905 ]
 [4.705918  4.71267   4.7136083]
 [4.673727  4.6809044 4.6816516]
 [4.67431   4.6814265 4.682199 ]
 [4.288162  4.2940116 4.2947083]
 [4.2168603 4.224157  4.224138 ]
 [4.5540943 4.5609703 4.5617714]
 [3.8867586 3.8902423 3.8909461]
 [4.2823925 4.288555  4.2891245]
 [4.2742424 4.2814384 4.2816224]
 [4.6733108 4.680466  4.681223 ]
 [4.1811123 4.188042  4.188039 ]
 [4.805822  4.813784  4.814254 ]
 [4.593412  4.5986753 4.600134 ]
 [4.6319523 4.638005  4.6391845]
 [4.709285  4.7163134 4.71714  ]
 [4.4742794 4.4810047 4.4817815]
 [4.4507885 4.4556417 4.457079 ]
 [4.5500517 4.5573683 4.5579925]
 [4.674381  4.68

## Grid Search

The hyperparamter tuning for NN.

In [ ]:
# # tolerance_value = 50
# from sklearn.metrics import make_scorer

# # coefficient of determination (R^2) for regression  (only for Keras tensors)
# def r_square(y_true, y_pred):
# #     y_true = tf.convert_to_tensor(value=y_true, dtype='float32')
   
#     SS_res =  K.sum(K.square(y_true - y_pred)) 
#     SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
#     return (1 - SS_res/(SS_tot + K.epsilon())).numpy()

# param_distribs = {
# #     "n_hidden": [4, 5, 6, 7, 8],
# #     "n_neurons": [1500, 2000, 2500, 3000, 3500, 4000],
#     "n_hidden": [5],
#     "n_neurons": [2500, 3000, 3500, 4000],
# }

# search_cv = GridSearchCV(estimator= keras_reg,
#                          param_grid= param_distribs, 
#                          cv= 4,
#                          scoring =make_scorer(r_square))

# grid_result = search_cv.fit(X_scaled, y.to_numpy())
# #                 validation_data=(X_test_scaled, y_test),
#                 # callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=tolerance_value)])

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']

# for mean, stdev, param in zip(means, stds, params):
#     print("mean: %f, std: (%f) parameters: %r" % (mean, stdev, param))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# best_model = grid_result.best_estimator_

# print ("\nHistory object\n")
# best_model_history = best_model.fit(X_train_scaled, y_train, epochs=n_epochs,
#               validation_data=(X_test_scaled, y_test),
#               callbacks=[keras.callbacks.EarlyStopping(patience=tolerance_value)])

# y_pred = best_model.predict(X_test)              

# plt.figure()
# plt.plot(best_model_history.history['loss'])
# plt.plot(best_model_history.history['val_loss'])
# plt.title("Model loss")
# plt.ylabel("Mean Square Error (MSE) - Loss")
# plt.xlabel("Epoch")
# plt.legend(["Train", "Test"])
# plt.show()

# print ("\n Best Model's Parameters")
# print (best_model.get_params())
# print ("\n The mean square error (MSE)")
# print (best_model.score(X_test, y_test))
# print ("\n The R-square metric is")
# print (r_square(tf.convert_to_tensor(value=y_test.values, dtype='float32'), y_pred))
# print ("\n The predicted values (Lattice constants)")
# print (y_pred)

RedirectMissingLocation: ignored